# Writing a Molecular Monte Carlo Simulation

Starting today, make sure you have the functions

1. `calculate_LJ` - written in class
1. `read_xyz` - provided in class
1. `calculate_total_energy` - modified version provided in this notebook written for homework which has cutoff
1. `calculate_distance` - should be the version written for homework which accounts for periodic boundaries.
1. `calculate_tail_correction` - written for homework 


In [1]:
# add imports here

import math

import random


In [2]:
def calculate_total_energy(coordinates, box_length, cutoff):
    """
    Calculate the total energy of a set of particles using the Lennard Jones potential.
    
    Parameters
    ----------
    coordinates : list
        A nested list containing the x, y,z coordinate for each particle
    box_length : float
        The length of the box. Assumes cubic box.
    cutoff : float
        The cutoff length
    
    Returns
    -------
    total_energy : float
        The total energy of the set of coordinates.
    """
    
    total_energy = 0
    num_atoms = len(coordinates)

    for i in range(num_atoms):
        for j in range(i+1, num_atoms):
            # Calculate the distance between the particles - exercise.
            dist_ij = calculate_distance(coordinates[i], coordinates[j], box_length)

            if dist_ij < cutoff:
                # Calculate the pairwise LJ energy
                LJ_ij = calculate_LJ(dist_ij)

                # Add to total energy.
                total_energy += LJ_ij
    return total_energy

def read_xyz(filepath):
    """
    Reads coordinates from an xyz file.
    
    Parameters
    ----------
    filepath : str
       The path to the xyz file to be processed.
       
    Returns
    -------
    atomic_coordinates : list
        A two dimensional list containing atomic coordinates
    """
    
    with open(filepath) as f:
        box_length = float(f.readline().split()[0])
        num_atoms = float(f.readline())
        coordinates = f.readlines()
    
    atomic_coordinates = []
    
    for atom in coordinates:
        split_atoms = atom.split()
        
        float_coords = []
        
        # We split this way to get rid of the atom label.
        for coord in split_atoms[1:]:
            float_coords.append(float(coord))
            
        atomic_coordinates.append(float_coords)
        
    return atomic_coordinates, box_length

def calculate_LJ(r_ij):
    """
    The LJ interaction energy between two particles.

    Computes the pairwise Lennard Jones interaction energy based on the separation distance in reduced units.

    Parameters
    ----------
    r_ij : float
        The distance between the particles in reduced units.
    
    Returns
    -------
    pairwise_energy : float
        The pairwise Lennard Jones interaction energy in reduced units.

    Examples
    --------
    >>> calculate_LJ(1)
    0

    """
    
    r6_term = math.pow(1/r_ij, 6)
    r12_term = math.pow(r6_term, 2)
    
    pairwise_energy = 4 * (r12_term - r6_term)
    
    return pairwise_energy


## Add your group's modified calculate_distance function
def calculate_distance(coord1, coord2, box_length=None):
    """
    Calculate the distance between two points. When `box_length` is set, the minimum image convention is used to calculate the distance between the points.

    Parameters
    ----------
    coord1, coord2 : list
        The coordinates of the points, [x, y, z]
    
    box_length : float, optional
        The box length. This function assumes box is a cube.

    Returns
    -------
    distance : float
        The distance between the two points.
    """
    
    distance = 0
    
    for i in range(3):
        dim_dist = (coord1[i] - coord2[i])
        if box_length:
            dim_dist = (dim_dist - box_length * round(dim_dist/ box_length))**2
        distance += dim_dist
        
        
    distance = math.sqrt(distance)
    
    return distance


## Add your group's tail correction function
import os

config1_file = os.path.join("lj_sample_configurations", "lj_sample_config_periodic1.txt")

sample_coords, box_length = read_xyz(config1_file)

print(box_length)

print(len(sample_coords))

def calculate_tail_correction(n,b,r_c):
    r3_term = math.pow(1/r_c, 3)
    r9_term = (1/3) * (math.pow(r3_term, 9))
    NV_term = (8*math.pi/3) * ((n**2)/(b**3))
    tail_correction = NV_term * (r9_term - r3_term)
    
    return tail_correction

10.0
800




Metropolis criterian 


P (m--> n) = min[1,e^-bdU]

In [3]:
def accept_or_reject(delta_U, beta):
    """
    
    
    
    
    
    Accept or reject a move based on metropolis criterion.
    
    Paramaters
    ----------------
    delta_U : float
    
        change in energy for moving systems from m to n
    
    
    
    beta : float
        1/temperature
    
    
    
    Returns
    --------
    bool
        whether move is accepted-true
        
    
        
    
    
    
    
    
    """
    
    
    if delta_U <= 0.0:
        accept = True
    else:
        #Gen random number on(0,1)
        random_number = random.random()
        p_acc = math.exp(-beta*delta_U)
        
        if random_number < p_acc:
            accept = True
         
        else:
            accept = False
            
    return accept


In [4]:
# Sanity checks - test cases

delta_energy = -1
beta = 1

accepted = accept_or_reject(delta_energy, beta)

assert accepted is True

In [5]:
delta_energy = 0
accepted = accept_or_reject(delta_energy, beta)
assert accepted is True

In [6]:
# To test function w/random numbers
# can set random seed

# To set seed

random.seed(0)

In [7]:
random.random()

0.8444218515250481

In [8]:
# Test case for positive delta energy

delta_energy = 1
beta = 1
random.seed(0)

accepted = accept_or_reject(delta_energy, beta)

assert accepted is False

In [9]:
random.seed(1)
accepted = accept_or_reject(delta_energy, beta)
assert accepted is True

In [10]:
#unset seed
random.seed()

In [11]:
def calculate_pair_energy(coordinates, i_particle, box_length, cutoff):
    
    """
    
    Calculate interaction eenrgy of particle w/ its environment (all other particles in sys)
    
    Parameters
    ----------------
    coordinates : list
        the coordinates for all particles in sys
    i_particle : int
        particle number for which to calculate energy
        
    cutoff : float
        simulation cutoff. beyond distnaces, interactions aren't calculated
    
    box length : float
    
        length of simultion box. assumse cubic boc
        
    
    Returns
    ---------------
    
    float
        pairwise interaction energy of ith particle w/all other particles in sys
        
    
    
    
    
    
    """
    
    e_total = 0.0
    i_position = coordinates[i_particle]
    
    num_atoms = len(coordinates)
    
    for j_particle in range(num_atoms):
        if i_particle != j_particle:
            
            j_position = coordinates[j_particle]
            rij = calculate_distance(i_position, j_position, box_length)
            
            if rij < cutoff:
                e_pair = calculate_LJ(rij)
                e_total += e_pair
                
                

    return e_total



In [12]:
## sanity checks
test_coords = [[0,0,0], [0, 0, 2**(1/6)], [0 ,0 , 2*2**(1/6)]]

#what do you expect results to be for particle index 1 *use cutoff of 3)

assert calculate_pair_energy(test_coords, 1, 10, 3) == -2

# what do you expect result for particle index w/cuttoff of 2?

assert calculate_pair_energy(test_coords, 0, 10, 2) == -1

assert calculate_pair_energy(test_coords, 0, 10, 3) == calculate_pair_energy(test_coords, 2, 10, 3)




## Monte carlo loop


In [13]:
#Read or generate inital coordiantes
coordinates, box_length = read_xyz('lj_sample_configurations/lj_sample_config_periodic1.txt')
# Set simulation parameters
reduced_temperature = 0.9

num_steps = 5000

max_displacement = 0.1

cutoff = 3

freq = 1000

# Calculated quantities

beta = 1 / reduced_temperature
num_particles = len(coordinates)

# Energy calculations 

total_energy = calculate_total_energy(coordinates, box_length, cutoff)
print(total_energy)

tail_correction = calculate_tail_correction(num_particles, box_length, cutoff)

print(tail_correction)

total_energy += tail_correction

for step in range(num_steps):
        
        # 1. Randomly pick one of particles
        random_particle = random.randrange(num_particles)
        
        # 2. Calculate in interavtion energy of sleectd particle w/system and store this value
        current_energy = calculate_pair_energy(coordinates, random_particle, box_length, cutoff)
       
        # 3. Gnerate random x,y,z displacement
        
        x_rand = random.uniform(-max_displacement, max_displacement)
        y_rand = random.uniform(-max_displacement, max_displacement)
        z_rand = random.uniform(-max_displacement, max_displacement)


        #4. Modify coordinate of Nth particle by genrated displacements
        coordinates[random_particle][0] += x_rand
        coordinates[random_particle][1] += y_rand
        coordinates[random_particle][2] += z_rand
        
        #5. Calculate interaction energy of moved particle w/system
        proposed_energy = calculate_pair_energy(coordinates, random_particle, box_length, cutoff)
        delta_energy = proposed_energy - current_energy
        #6. Calculate if we accept move based on energy diff
        
        accept = accept_or_reject(delta_energy, beta)
        #7. if accept, move particle
        if accept: 
            total_energy += delta_energy
        else:
            # Move is not accepted, roll back coordinates
            coordinates[random_particle][0] -= x_rand
            coordinates[random_particle][1] -= y_rand
            coordinates[random_particle][2] -= z_rand


        #8. print energy if step is multiple of freq
        
        if step % freq == 0:
            print(step, total_energy/num_particles)

-4351.540194543858
-198.57968378223148
0 -5.687649847907612
1000 -5.699617780370584
2000 -5.686182062849184
3000 -5.665588805681563
4000 -5.666167110628675
